In [ ]:
#| default_exp general

# General Functions

The functions in this module are not specific to any of the ITTC appendices

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd
from fastcore.test import *

## Conversion between knots and meters per second

As many of the functions in this library require SI units it can be helpful to have a knots-m/s converter and vice-versa.

In [ ]:
#| export
def knots_to_ms(knots:float #the speed in knots
               ) -> float: #speed in m/s
    
    "convert knots to m/s"
    
    return knots/1.943844

In [ ]:
#| export
def ms_to_knots(ms:float #the speed in m/s
               ) -> float: #speed in knots
    
    "convert knots to m/s"
    
    return ms * 1.943844

Using these simple functions we can easily convert between knots and m/s

In [ ]:
#convert knots to m/s
meters_per_second = knots_to_ms(5)
print('speed converted to meters per second '+str(meters_per_second))
#convert back to knots
knots = ms_to_knots(meters_per_second)
print('speed converted back into knots '+str(knots))

speed converted to meters per second 2.572222873852017
speed converted back into knots 5.0


In [ ]:
#| hide
#check the two functions are invserse of each other
test_eq(ms_to_knots(knots_to_ms(5)),5)

## Air resistance

The resistance experienced by the ship as a result of passing through air.

equivalent to equation 9 of ITTC

In [ ]:
#| export
def air_resistance(air_density:float, #Air density [kg/$m^3$]
                   wind_resistance_coef_rel:float, #the coefficient of wind resistance using the relative angle of the wind
                   wind_resistance_coef_zero:float, #the coefficient of wind resistance using angle 0 radians
                   area:float, #The maximum transverse area of the ship exposed to the wind [m^2]
                   relative_wind_speed:float, #Relative wind speed [m/s]
                   sog:float #speed over ground [m/s]
                   
                  )->float: #Air resistance [N]
    "Calculates the air resistance. N.B. SI units must be used. Do not use knots"
    
    wind_resistance = 0.5*air_density*area*(wind_resistance_coef_rel*relative_wind_speed**2 - wind_resistance_coef_zero*sog**2)
    
    return wind_resistance

What is the air resistance experienced by a cruise ferry with a tranverse area of 500$m^2$ travelling at 20 knots with a relative wind of 10 knots and a relative wind angle of $45^\circ$

In [ ]:
sog_ms = knots_to_ms(20)
wind_speed_ms = knots_to_ms(10)

air_resistance(1.2, 0.34, 0.69, 500, wind_speed_ms, sog_ms)

-19213.8238090769

In [ ]:
#| hide
#Function needs some test 

In [ ]:
#| hide
#what test should I put in here?
sog_ms = knots_to_ms(20)
wind_speed_ms = knots_to_ms(10)
test_close(air_resistance(1.2, 0.34, 0.69, 500, wind_speed_ms, sog_ms),-19213, eps = 1e0 )

## Resistance due to water temperature and salinity

The temperature and salt content of the water affect its density. As such, they also affect the ship resistance. The below function taken from section 10.3.3 of the ITTC corrects the sea water temperature, and salt content to be that of the appropriate reference values. 

| variable    | reference value     |
|-------------|---------------------|
| temperature | $15^\circ \text{C}$ |
| Density     | 1026$kg/m^3$        |

The coefficients of roughness used in the function are derived from ITTC Recommended Procedures 7.5-02-03-01.4

In [ ]:
#| export
def temp_salinity_water_resistance(CF:float, #frictional resistance coefficient for actual water temperature and salinity
                                   CF0:float, #frictional resistance coefficient for reference water temperature and salinity
                                   delta_CF:float, #roughness allowance associated with Reynolds number for actual water temperature and salinity
                                   delta_CF0:float,#roughness allowance associated with Reynolds number for reference water temperature and salinity
                                   CT0:float, #total resistance coefficient for reference water temperature and salinity
                                   #RF:float, #frictional resistance for actual water temperature and salt content [N]
                                   #RF0:float, #frictional resistance for reference water temperature and salt content [N]
                                   #RT0:float, #total resistance for reference water temperature and salt content [N]
                                   S:float, #wetted surface area [m2]
                                   VS:float, #ship’s speed through the water [m/s]
                                   rho_S:float, #water density for actual water temperature and salt content [kg/m3 ]
                                   rho_0:float = 1026 #water density for reference water temperature and salt content                                  
                                  )-> float: #resistance increase due to deviation of water temperature and water density [N]
    
    "Resistance due to water temperature and salinity corrected relative to the reference values"
    
    #This function can be re-written into sub-parts and all the sub-parts ouputted in a results matrix. 
    #Would this be useful?
    RF = 0.5 * rho_S* S * VS**2 * (CF + delta_CF)
    RT0 = 0.5 * rho_0 * S * VS**2 * CT0
    
    RAS =  RT0 * ( rho_S/rho_0 - 1 ) - RF* ( (CF0 + delta_CF0)/(CF + delta_CF) - 1 )
    
    return RAS
    

I have absolutely no idea what sensible values should look like, so this function will need to wait for beta testing, before the documentation can be complete

Also this function will probably need to point to some other functions for calculating the roughness coefficents.

In [ ]:
temp_salinity_water_resistance(1.2, 1, 0.2, 0.2, 1.4e-3, 2000, 10, 1050, 1026)

21003359.99999999

In [ ]:
show_doc(temp_salinity_water_resistance)

---

### temp_salinity_water_resistance

>      temp_salinity_water_resistance (CF:float, CF0:float, delta_CF:float,
>                                      delta_CF0:float, CT0:float, S:float,
>                                      VS:float, rho_S:float, rho_0:float=1026)

Resistance due to water temperature and salinity corrected relative to the reference values

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| CF | float |  | frictional resistance coefficient for actual water temperature and salinity |
| CF0 | float |  | frictional resistance coefficient for reference water temperature and salinity |
| delta_CF | float |  | roughness allowance associated with Reynolds number for actual water temperature and salinity |
| delta_CF0 | float |  | roughness allowance associated with Reynolds number for reference water temperature and salinity |
| CT0 | float |  | total resistance coefficient for reference water temperature and salinity |
| S | float |  | wetted surface area [m2] |
| VS | float |  | ship’s speed through the water [m/s] |
| rho_S | float |  | water density for actual water temperature and salt content [kg/m3 ] |
| rho_0 | float | 1026 | water density for reference water temperature and salt content |
| **Returns** | **float** |  | **resistance increase due to deviation of water temperature and water density [N]** |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()